# Simulating lightcurves using an expanding black body model

In [1]:
import os
home_dir = os.environ.get('HOME')

# Please enter the filename of the ztf_sim output file you would like to use. The example first determines
# your home directory and then uses a relative path (useful if working on several machines with different usernames)
survey_file = os.path.join(home_dir, 'data/ZTF/test_schedule_v6.db')

# Please enter the path to where you have placed the Schlegel, Finkbeiner & Davis (1998) dust map files
# You can also set the environment variable SFD_DIR to this path (in that case the variable below should be None)
sfd98_dir = os.path.join(home_dir, 'data/sfd98')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.cosmology import Planck15
import simsurvey
import sncosmo

import simsurvey_tools as sst

In [3]:
# Load the ZTF CCD corners and filters
ccds = sst.load_ztf_ccds()
sst.load_ztf_filters()

In [4]:
# Load simulated survey from file (download from ftp://ftp.astro.caltech.edu/users/ebellm/one_year_sim_incomplete.db)
plan = simsurvey.SurveyPlan(load_opsim=survey_file, band_dict={'g': 'ztfg', 'r': 'ztfr', 'i': 'desi'}, ccds=ccds)

mjd_range = (plan.cadence['time'].min() - 30, plan.cadence['time'].max() + 30)

In [5]:
# The expanding black body model takes arguments that define the evolution of the transient (functions + parameters).
# This example is the black body fit to GW170817 from Kasliwal et al. (2017).

tr_prop = {
    'lcmodel_prop': dict(
        minphase=0.1,
        maxphase=15.,
        tempfunc=(lambda p, x: p[0] * x**p[1]),
        tempparam=[6050, -0.62],
        radiusfunc=(lambda p, x: p[0]*(1-np.exp(-p[1]*x)) + p[2]*x),
        radiusparam=[24000, 0.42, 2500]
    )
}

tr = simsurvey.get_transient_generator((0., 0.05),
                                       ratefunc=lambda z: 5e-5,
                                       transient='generic', 
                                       template='ExpandingBlackBody',
                                       dec_range=(-30,90),
                                       mjd_range=(mjd_range[0],
                                                  mjd_range[1]),
                                       sfd98_dir=sfd98_dir,
                                       transientprop=tr_prop)

In [6]:
survey = simsurvey.SimulSurvey(generator=tr, plan=plan)
    
lcs = survey.get_lightcurves(
    #progress_bar=True, notebook=True # If you get an error because of the progress_bar, delete this line.
)

/home/ufeindt/anaconda2/envs/simsurvey-dev/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in double_scalars


In [7]:
print len(lcs.lcs)

25


In [8]:
lcs[0]

time,band,flux,fluxerr,zp,zpsys,field,ccd,comment
float64,str4,float64,float64,int64,str2,int64,int64,unicode7
57638.49337179176,ztfg,355.32161087553584,528.2578611868671,30,ab,781,5,all_sky
57638.50834715247,ztfr,496.6356884816409,585.6689373811489,30,ab,781,5,all_sky
57639.49319883449,ztfg,-1112.7071252858323,540.88848202504,30,ab,781,5,all_sky
57639.517170348794,ztfr,221.48953614664978,569.1856342438408,30,ab,781,5,all_sky
57643.49337246782,ztfg,608.910422918301,518.6086501956214,30,ab,781,5,all_sky
57647.48978592098,ztfr,-1300.7760579660967,1188.463450023813,30,ab,781,5,all_sky
57647.49024888394,ztfr,2258.417624555949,1188.463450023813,30,ab,781,5,all_sky
57647.51940756444,ztfg,442.12984942724836,933.9906344607986,30,ab,781,5,all_sky
57648.451528856065,ztfr,173.89108738002778,1312.8618893003395,30,ab,781,5,all_sky
